In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import sys
import os
import math
from data import load_NEMSIS_dask, load_NEMSIS_pandas, chain_join

In [2]:
# data_dir: os.PathLike = "/Volumes/Extreme Pro/datasets/NEMSIS/ASCII/"
# data_dir: os.PathLike = "/media/jbajor/Extreme Pro/datasets/NEMSIS/ASCII/"
data_dir: os.PathLike = "/home/jbajor/Dev/datasets/NEMSIS/ASCII/"


nemsis_subset = {
    "ComputedElements":[
        'PcrKey',
        'NasemsoRegion',
        'Urbanicity',
        'ageinyear',
        'EMSDispatchCenterTimeSec',
        'EMSChuteTimeMin',
        'EMSSceneResponseTimeMin',
        'EMSSceneTimeMin',
        'EMSSceneToPatientTimeMin'
    ],
    "FACTPCRADDITIONALRESPONSEMODE":None,
    "FACTPCRADDITIONALSYMPTOM":None,
    "FACTPCRADDITIONALTRANSPORTMODE":None,
    "FACTPCRARRESTCPRPROVIDED":None,
    "FACTPCRARRESTRESUSCITATION":None,
    "FACTPCRARRESTRHYTHMDESTINATION":None,
    "FACTPCRARRESTROSC":None,
    "FACTPCRARRESTWITNESS":None,
    "FACTPCRBARRIERTOCARE":None,
    "FACTPCRMEDICATION":None,
    "FACTPCRPRIMARYIMPRESSION":None,
    "FACTPCRPROCEDURE":None,
    "FACTPCRPROTOCOL":None,
    "FACTPCRSECONDARYIMPRESSION":None,
    "FACTPCRTRAUMACRITERIA":None,
    "FACTPCRWORKRELATEDEXPOSURE":None,
    # "PCRMEDCOMPGROUP":None, # bad key
    # "PCRPROCCOMPGROUP":None, # bad key
    # "PCRVITALECGGROUP":None, # bad key
    "Pub_PCRevents":[
        'PcrKey',
        'eArrest_01',
        'eArrest_02',
        'eArrest_05',
        'eArrest_07',
        'eArrest_11',
        'eArrest_14',
        'eArrest_16',
        'eArrest_18',
        'eDisposition_23',
        'ePayment_01',
        'eResponse_07',
        'eResponse_15',
        'eSituation_01',
        'eSituation_07',
        'eSituation_08',
        'eTimes_01',
        'eTimes_03',
        'eTimes_05',
        'eTimes_06',
        'eTimes_07',
        'eTimes_09',
        'eTimes_11',
        'eTimes_12',
        'eTimes_13'
    ],
}


In [3]:
dataframes = load_NEMSIS_pandas(data_dir=data_dir, load_targets=nemsis_subset, nrows=1000000)

In [4]:
joined = chain_join(datadict=dataframes, key='PcrKey', start_table='Pub_PCRevents', type='left')

In [5]:
joined.to_pickle('NEMSIS_subset.pkl')